In [1]:
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
import warnings
warnings.filterwarnings('ignore')

In [2]:
dadosFreq = pd.read_excel("Dados originais_artigo Priscila .xlsx")

In [3]:
dadosFreq.head()

,Espécie - Abelha,Freq. 1 (Hz),Freq. 2 (Hz),Freq. 3 (Hz),Média Freq.,Peso abelha (g),Tamanho tórax,Duração (s),Média Freq..1,Peso abelha (g).1,Tamanho tórax.1,Duração (s).1
0,Augochloropsis_sp1,219.4,197.3,203.7,206.800000,0.0608,2.1,4.1780,15.62052,0.007294,0.122474,1.570395
1,Augochloropsis_sp1,206.4,205.1,216.0,209.166667,0.0608,2.1,1.6670,NaN,NaN,NaN,NaN
2,Augochloropsis_sp1,187.3,186.6,204.0,192.633333,0.0448,1.9,0.5803,NaN,NaN,NaN,NaN
3,Augochloropsis_sp1,234.7,229.3,214.8,226.266667,0.0534,1.9,0.7520,NaN,NaN,NaN,NaN
4,Augochloropsis_sp1,214.6,194.8,205.8,205.066667,0.0448,1.9,NaN,NaN,NaN,NaN,NaN


In [4]:
dados = dadosFreq.drop(["Peso abelha (g)", "Tamanho tórax", "Duração (s)", "Média Freq..1",\
                            "Peso abelha (g).1", "Tamanho tórax.1", "Duração (s).1"], axis=1)

In [5]:
dados.head()

,Espécie - Abelha,Freq. 1 (Hz),Freq. 2 (Hz),Freq. 3 (Hz),Média Freq.
0,Augochloropsis_sp1,219.4,197.3,203.7,206.800000
1,Augochloropsis_sp1,206.4,205.1,216.0,209.166667
2,Augochloropsis_sp1,187.3,186.6,204.0,192.633333
3,Augochloropsis_sp1,234.7,229.3,214.8,226.266667
4,Augochloropsis_sp1,214.6,194.8,205.8,205.066667


In [23]:
X = list(dados['Média Freq.'])
y = list(dados['Espécie - Abelha'])

In [25]:
for i in range(len(X)):
    X[i] = [X[i]]
print(X)

[[206.8], [209.166666666667], [192.633333333333], [226.266666666667], [205.066666666667], [180.3], [190.1], [296.466666666667], [207.9], [257.1], [232.566666666667], [214.133333333333], [227.433333333333], [264.1], [208.533333333333], [235.3], [208.3], [330.80808080808], [312.033333333333], [327.933333333333], [196.733333333333], [188.233333333333], [175.933333333333], [118.733333333333], [175.866666666667], [158.6], [175.9], [169.866666666667], [158.3], [303.666666666667], [351.833333333333], [341.6], [318.7], [371.7], [366.433333333333], [298.433333333333], [345.2], [324.6], [330.6], [314.833333333333], [322.266666666667], [250.666666666667], [243.833333333333], [279.3], [221.033333333333], [204.266666666667], [220.033333333333], [207.2], [223.766666666667], [250.866666666667], [214.166666666667], [190.133333333333], [219.2], [205.433333333333], [182.433333333333], [153.333333333333], [153.8], [193.366666666667], [148.3], [136.333333333333], [147.166666666667]]


In [14]:
def salva_modelo(modelo, nome_arq):
    filename = '/home/alison/Documentos/Projeto-Tomate/modelos/' + nome_arq
    pickle.dump(modelo, open(filename, 'wb'))


In [15]:
def treino_teste(matriz, classes, clf):
    X_train, X_test, y_train, y_test = train_test_split(matriz, classes, test_size=0.5, random_state=1)
    #kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

    clf.fit(X_train, y_train)

    y_true, y_pred = y_test, clf.predict(X_test)
    
    print("Acurácia...: %.4f" %(metrics.accuracy_score(y_true, y_pred) * 100))
    print("Precision..: %.4f" %(metrics.precision_score(y_true, y_pred, average='macro') * 100))
    print("Recall.....: %.4f" %(metrics.recall_score(y_true, y_pred, average='macro') * 100))
    print("F1-Score...: %.4f" %(metrics.f1_score(y_true, y_pred, average='macro') * 100))
    print()
    print(metrics.classification_report(y_true, y_pred))

    return clf

In [16]:
def algoritmos(op, matriz, classes):

    if op == "svm":
        modelo = SVC(C=10, decision_function_shape='ovo', gamma=0.01, kernel='rbf')
        
        clf = treino_teste(matriz, classes, modelo)

        modelname = "baseline_svm_.sav"
        salva_modelo(clf, modelname)

    elif op == "lr":
        modelo = LogisticRegression(C=10)
        clf = treino_teste(matriz, classes, modelo)

        modelname = "baseline_lr_.sav"
        salva_modelo(clf, modelname)

    elif op == "dtree":
        modelo = DecisionTreeClassifier(criterion='entropy', max_depth=10, min_samples_split=2)
        clf = treino_teste(matriz, classes, modelo)

        modelname = "baseline_dtree_.sav"
        salva_modelo(clf, modelname)

    elif op == "rf":
        modelo = RandomForestClassifier(max_features='sqrt', n_estimators=100)
        clf = treino_teste(matriz, classes, modelo)

        modelname = "baseline_rf_.sav"
        salva_modelo(modelo, modelname)

    elif op == "ens":
        svc = SVC(C=10)
        rf = RandomForestClassifier()
        lr = LogisticRegression(C=1)

        modelos = [('svc', svc), ('rf', rf), ('lr', lr)]

        votingclf = VotingClassifier(estimators=modelos, voting='hard')
        clf = treino_teste(matriz, classes, votingclf)

        modelname = "baseline_ensemble_.sav"
        salva_modelo(clf, modelname)

In [26]:
algoritmo = "svm"
algoritmos(algoritmo, X, y)

Acurácia...: 35.4839
Precision..: 26.5385
Recall.....: 24.3590
F1-Score...: 24.0293

                                precision    recall  f1-score   support

            Augochloropsis_sp1       0.00      0.00      0.00         2
            Augochloropsis_sp2       0.00      0.00      0.00         0
                  Bombus_morio       0.00      0.00      0.00         6
             Bombus_pauloensis       0.20      0.50      0.29         2
               Centris_tarsata       0.00      0.00      0.00         1
           Centris_trigonoides       0.00      0.00      0.00         1
            Exomalopsis_analis       0.75      0.50      0.60         6
             Exomalopsis_minor       1.00      0.50      0.67         2
              Melipona_bicolor       1.00      1.00      1.00         4
       Melipona_quadrifasciata       0.50      0.67      0.57         3
Pseudoalglochloropsis_graminea       0.00      0.00      0.00         2
          Xylocopa_nigrocincta       0.00      0.0

In [91]:
algoritmo = "lr"
algoritmos(algoritmo, X, y)

Acurácia...: 16.1290
Precision..: 4.7619
Recall.....: 8.9744
F1-Score...: 6.1538

                                precision    recall  f1-score   support

  Augochloropsis_brachycephala       0.00      0.00      0.00         0
            Augochloropsis_sp1       0.00      0.00      0.00         2
                  Bombus_morio       0.00      0.00      0.00         6
             Bombus_pauloensis       0.33      0.50      0.40         2
               Centris_tarsata       0.00      0.00      0.00         1
           Centris_trigonoides       0.00      0.00      0.00         1
            Exomalopsis_analis       0.29      0.67      0.40         6
             Exomalopsis_minor       0.00      0.00      0.00         2
              Melipona_bicolor       0.00      0.00      0.00         4
       Melipona_quadrifasciata       0.00      0.00      0.00         3
Pseudoalglochloropsis_graminea       0.00      0.00      0.00         2
          Xylocopa_nigrocincta       0.00      0.00  

In [92]:
algoritmo = "rf"
algoritmos(algoritmo, X, y)

Acurácia...: 9.6774
Precision..: 7.5092
Recall.....: 7.6923
F1-Score...: 7.3373

                                precision    recall  f1-score   support

  Augochloropsis_brachycephala       0.00      0.00      0.00         0
            Augochloropsis_sp1       0.00      0.00      0.00         2
                  Bombus_morio       0.00      0.00      0.00         6
             Bombus_pauloensis       0.00      0.00      0.00         2
               Centris_tarsata       0.00      0.00      0.00         1
           Centris_trigonoides       0.00      0.00      0.00         1
            Exomalopsis_analis       0.14      0.17      0.15         6
             Exomalopsis_minor       0.33      0.50      0.40         2
              Melipona_bicolor       0.00      0.00      0.00         4
       Melipona_quadrifasciata       0.50      0.33      0.40         3
Pseudoalglochloropsis_graminea       0.00      0.00      0.00         2
          Xylocopa_nigrocincta       0.00      0.00   

In [93]:
algoritmo = "dtree"
algoritmos(algoritmo, X, y)

Acurácia...: 6.4516
Precision..: 4.7009
Recall.....: 3.8462
F1-Score...: 4.1026

                                precision    recall  f1-score   support

  Augochloropsis_brachycephala       0.00      0.00      0.00         0
            Augochloropsis_sp1       0.00      0.00      0.00         2
                  Bombus_morio       0.00      0.00      0.00         6
             Bombus_pauloensis       0.00      0.00      0.00         2
               Centris_tarsata       0.00      0.00      0.00         1
           Centris_trigonoides       0.00      0.00      0.00         1
            Exomalopsis_analis       0.11      0.17      0.13         6
             Exomalopsis_minor       0.00      0.00      0.00         2
              Melipona_bicolor       0.00      0.00      0.00         4
       Melipona_quadrifasciata       0.50      0.33      0.40         3
Pseudoalglochloropsis_graminea       0.00      0.00      0.00         2
          Xylocopa_nigrocincta       0.00      0.00   

In [94]:
algoritmo = "ens"
algoritmos(algoritmo, X, y)

Acurácia...: 22.5806
Precision..: 9.3254
Recall.....: 18.0556
F1-Score...: 12.0833

                                precision    recall  f1-score   support

  Augochloropsis_brachycephala       0.00      0.00      0.00         0
            Augochloropsis_sp1       0.00      0.00      0.00         2
                  Bombus_morio       0.00      0.00      0.00         6
             Bombus_pauloensis       0.17      0.50      0.25         2
               Centris_tarsata       0.00      0.00      0.00         1
           Centris_trigonoides       0.00      0.00      0.00         1
            Exomalopsis_analis       0.29      0.67      0.40         6
             Exomalopsis_minor       0.67      1.00      0.80         2
              Melipona_bicolor       0.00      0.00      0.00         4
       Melipona_quadrifasciata       0.00      0.00      0.00         3
Pseudoalglochloropsis_graminea       0.00      0.00      0.00         2
          Xylocopa_nigrocincta       0.00      0.00